<h1 style="color: #006400;">Creating a Machine Learning Model</h1>
<h3 style="color: #8b5e3c;">Applying machine learning to create labels for our data</h3>
<p style="color: #8b5e3c;">Mathias Galvan</p>

<h2 style="color: #006400;"> Loading the Data </h2>

In [1]:
import pandas as pd

